<a href="https://colab.research.google.com/github/Rojerr9241/3DGRAVITY/blob/main/Si_IC_00_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estimación de la Temperatura de la Superficie Terrestre

## Preámbulo

### 1. Librerías y Google Drive

In [ ]:
# @title
# --- 1. INSTALACIÓN E IMPORTACIONES NECESARIAS ---
import ee
import geemap
import geopandas as gpd
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import date, timedelta
import datetime
import calendar # Necesario para calcular días en el mes
import matplotlib.pyplot as plt
import os # Necesario para la lógica de copia de archivos
from pathlib import Path  # Biblioteca moderna para manejar rutas
from google.colab import files
import io
import csv
import re
import unicodedata

# --- 2. CONFIGURACIÓN ---
# Todas las variables que podrías necesitar cambiar están aquí.
# --- 2.1 PARÁMETROS DE IMPORTACIÓN DATOS ---
DRIVE_MOUNT_POINT = '/content/drive'
SHARED_DRIVE_NAME = "Si_IC (Isla de Calor CAMe)"
FOLDER_NAME = "Datos"
FILE_NAME = "megalopolis_mun.geojson"
REQUIRED_COLUMNS = ['NOM_ENT', 'NOM_MUN']
# --- 2.2 PARÁMETROS DE EXPORTACIÓN Y COPIA ---
# Cambia esta variable a True si deseas exportar los rásters a Google Drive
EXPORTAR_A_DRIVE = True
# Nombre de la carpeta en Google Drive donde GEE guarda los resultados (DEBE EXISTIR)
DRIVE_FOLDER = 'GEE_TSM_Exports'
# --- 2.3 PARÁMETROS GEE **MODIFICAR** ---
GEE_PROJECT_ID = 'ricardo-ochoa'

# --- 3. FUNCIONES CORE DE EARTH ENGINE ---

# Parámetros de Visualización para la TSM
VIS_PARAMS_TSM = {
    'min': 15,
    'max': 40,
    'palette': [
        '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
        '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
        '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
        'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
        'ff0000', 'de0101', 'c21301', 'a71001', '911001'
    ]
}

# --- 4. FUNCIONES DE UTILIDAD (HELPERS) ---
# 1. MOUNT DRIVE AND UPLOAD GDF
def check_is_colab() -> bool:
    """Verifica si el script se ejecuta en Google Colab."""
    try:
        import google.colab
        return True
    except ImportError:
        return False

def _mount_google_drive() -> bool:
    """
    Monta Google Drive si está en Colab y es necesario.
    Es 'idempotente': se puede llamar varias veces, pero solo montará una vez.

    Returns:
        bool: True si Drive está listo (ya montado o montado con éxito).
              False si el montaje falló.
    """
    mount_path = Path(DRIVE_MOUNT_POINT)

    # 1. Verificar si ya está montado
    if mount_path.is_dir() and any(mount_path.iterdir()):
        print(f"Google Drive ya está montado en {DRIVE_MOUNT_POINT}.")
        return True

    # 2. Si no, intentar montar
    try:
        from google.colab import drive
        print(f"Montando Google Drive en {DRIVE_MOUNT_POINT}...")
        drive.mount(DRIVE_MOUNT_POINT)
        print("¡Drive montado exitosamente!")
        return True
    except Exception as e:
        print(f"ERROR: Fallo al montar Google Drive: {e}")
        return False

def _read_geodataframe(file_path: Path) -> gpd.GeoDataFrame | None:
    """
    Intenta leer un archivo GeoJSON desde la ruta.
    Maneja los errores de I/O (Input/Output), como archivo no encontrado o corrupto.
    """
    if not file_path.exists():
        print(f"\nERROR: El archivo no se encontró en la ruta: {file_path}")
        return None

    try:
        print(f"\nCargando GeoJSON desde: {file_path}")
        gdf = gpd.read_file(file_path)
        print("¡GeoDataFrame cargado exitosamente!")
        return gdf
    except Exception as e:
        print(f"\nERROR al leer el archivo GeoJSON: {e}")
        return None

def _validate_gdf_columns(gdf: gpd.GeoDataFrame) -> bool:
    """
    Valida que el GeoDataFrame contenga las columnas requeridas.
    """
    current_columns = set(gdf.columns)
    required = set(REQUIRED_COLUMNS)
    missing_cols = required - current_columns

    if missing_cols:
        print(f"ERROR: El GeoJSON no contiene las columnas requeridas: {missing_cols}")
        return False

    print("Validación de columnas exitosa.")
    return True

# NEW
def exportar_imagen_a_drive(image, bands, roi, filename, folder):
    """
    Configura y comienza la tarea de exportación ASÍNCRONA a Google Drive.
    Se ejecutará automáticamente gracias al GEE_PROJECT_ID.

    Args:
        image (ee.Image): Imagen (ya clipeada) a exportar.
        bands (list): Lista de strings de las bandas a exportar (ej. ['LST_C'] o ['SR_B4', 'SR_B3', 'SR_B2']).
        roi (ee.Geometry): Región de interés (usada para la metadata 'region').
        filename (str): Nombre del archivo de salida.
        folder (str): Carpeta destino dentro de "My Drive".
    """
    print(f"  -> Iniciando exportación automática para: {filename}")
    task = ee.batch.Export.image.toDrive(
        image=image.select(bands).float(), # Selecciona las bandas y convierte a float
        description=filename,
        folder=folder,
        fileNamePrefix=filename,
        region=roi.bounds().getInfo()['coordinates'], # Usar la caja delimitadora de la ROI
        scale=30, # Resolución de Landsat
        crs='EPSG:4326',
        maxPixels=1e10
    )
    task.start()

    # Devuelve el mensaje de log para imprimirlo en la salida
    return f"  -> EXPORTACIÓN INICIADA: '{filename}'. (Monitorear en pestaña 'Tasks' de GEE)."

def limpiar_nombre_para_gee(nombre):
    """
    Limpia un string para que sea un nombre de archivo/descripción
    válido para GEE.
    Ej: 'Querétaro' -> 'Queretaro'
    Ej: 'Ciudad de México' -> 'Ciudad_de_Mexico'
    """
    # 1. Normalizar para separar acentos de letras (ej. 'é' -> 'e' + '´')
    nfkd_form = unicodedata.normalize('NFKD', nombre)

    # 2. Quedarse solo con los caracteres ASCII (ignora los acentos)
    solo_ascii = nfkd_form.encode('ASCII', 'ignore').decode('utf-8')

    # 3. Reemplazar espacios o guiones con guion bajo
    nombre_con_guiones = re.sub(r'[\s.-]+', '_', solo_ascii)

    # 4. Eliminar cualquier otro caracter que no sea letra, número o guion bajo
    nombre_limpio = re.sub(r'[^a-zA-Z0-9_]', '', nombre_con_guiones)

    return nombre_limpio

# DATA PROCESSING FUNCTIONS
def enmascarar_nubes(image):
    """
    Función para enmascarar nubes y sombras de nubes.
    """
    qa = image.select('QA_PIXEL')
    cloud_shadow_bit_mask = (1 << 3)
    cloud_bit_mask = (1 << 5)

    mask = qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0).And(
           qa.bitwiseAnd(cloud_bit_mask).eq(0))

    return image.updateMask(mask)

def enmascarar_nubes_scala(image):
    # --- 1. Calcular factores de escala ---
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBand = image.select('ST_B10').multiply(0.00341802).add(149.0)

    # --- 2. Tu máscara original ---
    qa = image.select('QA_PIXEL')
    cloud_shadow_bit_mask = (1 << 3)
    cloud_bit_mask = (1 << 5)
    mask = qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0).And(
           qa.bitwiseAnd(cloud_bit_mask).eq(0))

    # --- 3. Devolver imagen escalada Y enmascarada ---
    # ¡Tienes que usar addBands con 'True' para sobrescribir!
    return image.addBands(opticalBands, None, True) \
                .addBands(thermalBand, None, True) \
                .updateMask(mask)

def calcular_tsm(image, roi):
    """
    Función para calcular la Temperatura de la Superficie Terrestre (TSM) en Celsius.
    Requiere el 'roi' para calcular el min/max de NDVI en la región.
    """
    # 1. Seleccionar Bandas Ópticas y Térmicas y aplicar factores de escala
    optical_bands = image.select('SR_B4', 'SR_B5').multiply(0.0000275).add(-0.2)
    thermal_band = image.select('ST_B10').multiply(0.00341802).add(149.0)

    image_procesada = image.addBands(optical_bands.rename(['Red', 'NIR'])) \
                          .addBands(thermal_band.rename(['Brightness_Temp_K']))

    # 2. Calcular NDVI
    ndvi = image_procesada.normalizedDifference(['NIR', 'Red']).rename('NDVI')
    image_procesada = image_procesada.addBands(ndvi)

    # 3. Calcular Emisividad
    # Reducir para encontrar min/max de NDVI en la RDI
    min_max_ndvi = ndvi.reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=roi,
        scale=30,
        maxPixels=1e9
    )

    min_val = ee.Number(min_max_ndvi.get('NDVI_min'))
    max_val = ee.Number(min_max_ndvi.get('NDVI_max'))

    # --- CORRECCIÓN DE ERROR NULO (Añadido) ---
    # Si min_val es nulo (es decir, la imagen está completamente enmascarada/nublada en la ROI),
    # devolvemos la imagen sin la banda LST_C para que la mediana ignore esta imagen.
    es_valido = min_val.gt(-99999).And(max_val.gt(-99999)) # Usar una comprobación simple de no-nulo

    def calcular_y_añadir_tsm(min_val, max_val):
        # Fracción de Vegetación (PV)
        pv = ndvi.subtract(min_val).divide(max_val.subtract(min_val)).pow(2).rename('PV')

        # Emisividad
        emisividad = pv.multiply(0.004).add(0.986).rename('EMISSIVITY')

        # 4. Calcular la Temperatura de la Superficie Terrestre en Celsius
        brightness_temp_k = image_procesada.select('Brightness_Temp_K')
        lambda_term = ee.Image(0.00115).multiply(brightness_temp_k).divide(ee.Number(1.4388))

        tsm = brightness_temp_k.divide(
            ee.Image(1).add(
                lambda_term.multiply(emisividad.log())
            )
        ).subtract(273.15).rename('LST_C') # Kelvin a Celsius (-273.15)

        return image.addBands(tsm)

    # ee.Algorithms.If verifica si es_valido es True
    return ee.Algorithms.If(
        es_valido,
        calcular_y_añadir_tsm(min_val, max_val),
        image.select([]) # Si no es válido, devuelve una imagen vacía (o sin la banda TSM)
    )

# --- 3. FUNCIONES CORE DE EARTH ENGINE ---
# ... (Después de 'VIS_PARAMS_TSM', 'enmascarar_nubes', 'calcular_tsm', etc.)
def calcular_mediana_en_puntos(imagen, fc, escala):
    """
    Función helper robusta. Extrae el valor LST_C en cada punto de la
    colección 'fc' y luego calcula la MEDIANA de todos esos valores.
    Devuelve un número (float) o None si falla.
    """
    try:
        # 1. Usar reduceRegions para obtener el valor en CADA punto.
        # Esto devuelve una FeatureCollection donde cada Feature tiene
        # una propiedad 'median' con el valor LST de ese píxel.
        fc_con_valores = imagen.select('LST_C').reduceRegions(
            reducer=ee.Reducer.median(), # Reducer para muestrear el píxel
            collection=fc,
            scale=escala
        )

        # 2. Calcular la mediana DE ESOS VALORES (de la propiedad 'median')
        # ¡ESTA ES LA LÍNEA CORREGIDA!
        # Usamos reduceColumns para calcular la mediana de la *propiedad* 'median'
        # de toda la colección de puntos.
        mediana_de_puntos_dict = fc_con_valores.reduceColumns(
            reducer=ee.Reducer.median(),  # Reducer para la estadística
            selectors=['median']          # El nombre de la propiedad a reducir
        )

        # 3. Traer el valor a Python
        # El resultado es un dict, ej {'median': 25.5}
        mediana_num = ee.Number(mediana_de_puntos_dict.get('median')).getInfo()

        return mediana_num # Será un float o None

    except Exception as e:
        # Imprimir el error real de GEE es más útil
        print(f"Advertencia: Falló el cálculo de mediana en puntos. Error: {e}")
        return None

def calcular_score_de_confiabilidad(imagen_mediana, puntos_calientes_fc, puntos_frios_fc, escala=30):
    """
    Calcula el score de confiabilidad para una imagen de mediana (ráster final).
    Maneja todos los edge cases:
    - No hay puntos
    - Solo puntos fríos
    - Solo puntos calientes
    - Hay ambos (ideal)

    Devuelve: (score, log_mensaje)
    - score (float): El score numérico. Higher is better.
    - score (None): Si el score es No Aplicable (ej. no hay puntos).
    """

    hot_size = 0
    cold_size = 0

    # --- 1. Obtener tamaños (con .getInfo() para tomar decisiones en Python) ---
    try:
        if puntos_calientes_fc:
            hot_size = puntos_calientes_fc.size().getInfo()
        if puntos_frios_fc:
            cold_size = puntos_frios_fc.size().getInfo()
    except Exception as e:
        return -9999.0, f"Error al leer tamaño de puntos: {e}"

    # --- 2. Manejar Edge Cases ---

    # CASO 1: No hay puntos de control
    if hot_size == 0 and cold_size == 0:
        return None, "No hay puntos de control para calificar."

    # CASO 4: Ideal - Hay ambos tipos de puntos
    elif hot_size > 0 and cold_size > 0:
        mediana_caliente = calcular_mediana_en_puntos(imagen_mediana, puntos_calientes_fc, escala)
        mediana_fria = calcular_mediana_en_puntos(imagen_mediana, puntos_frios_fc, escala)

        if mediana_caliente is not None and mediana_fria is not None:
            # La física se cumple (caliente > frio)
            if mediana_caliente > mediana_fria:
                score = mediana_caliente - mediana_fria
                log = f"Delta Mediana (C-F): {score:.2f} ({mediana_caliente:.2f} - {mediana_fria:.2f})"
                return score, log
            # La física está invertida
            else:
                score = -1000.0 # Score de penalización
                log = f"Delta Invertido: {mediana_caliente:.2f} (Cal) <= {mediana_fria:.2f} (Frio)"
                return score, log
        else:
            return -9999.0, "Error: No se pudo extraer T° de píxeles en puntos."

    # CASO 3: Solo hay puntos calientes
    elif hot_size > 0 and cold_size == 0:
        mediana_caliente = calcular_mediana_en_puntos(imagen_mediana, puntos_calientes_fc, escala)
        if mediana_caliente is not None:
            # El score es la T° misma. Más caliente = mejor.
            score = mediana_caliente
            log = f"Plausibilidad Caliente: {score:.2f} (Solo puntos Cal)"
            return score, log
        else:
            return -9999.0, "Error: No se pudo extraer T° de píxeles en puntos calientes."

    # CASO 2: Solo hay puntos fríos
    elif cold_size > 0 and hot_size == 0:
        mediana_fria = calcular_mediana_en_puntos(imagen_mediana, puntos_frios_fc, escala)
        if mediana_fria is not None:
            # El score es la T° *negativa*.
            # Así, una T° más baja (ej. 15°C) da un score más alto (-15)
            # que una T° más alta (ej. 20°C, score -20).
            # "Higher is better" se mantiene.
            score = -mediana_fria
            log = f"Plausibilidad Fría (Score -T): {score:.2f} (T={mediana_fria:.2f})"
            return score, log
        else:
            return -9999.0, "Error: No se pudo extraer T° de píxeles en puntos fríos."

    # Por si acaso
    return -9999.0, "Caso de score no manejado."

def procesar_capa(coleccion_filtrada, roi, nombre_capa,
                   puntos_calientes_fc, puntos_frios_fc, # <-- Argumentos nuevos
                   visibilidad=False):
    """
    Aplica el procesamiento TSM, calcula la mediana, CALCULA EL SCORE
    e inicia la exportación.
    DEVUELVE la imagen, nombre, log y SCORE.
    """

    # ... (Tu lógica para 'tsm_coleccion' y la comprobación .size() se mantiene igual)
    # ... (tsm_coleccion = coleccion_filtrada.map(...)) ...
    tsm_coleccion = coleccion_filtrada.map(enmascarar_nubes).map(
        lambda img: calcular_tsm(img, roi)
    ).filter(ee.Filter.listContains('system:band_names', 'LST_C'))


    if tsm_coleccion.size().getInfo() == 0:
        # Si no hay imágenes, devolvemos None para la imagen y score fallido
        log_msg = f"  -> Capa '{nombre_capa}' OMITIDA: No hay imágenes válidas."
        return (None, nombre_capa, log_msg, -9999.0) # Score fallido

    # 1. Calcular mediana y CLIPAR UNA SOLA VEZ
    mediana_tsm = tsm_coleccion.select('LST_C').median().clip(roi)

    # 2. ¡AQUÍ INCORPORAMOS LOS PUNTOS!
    # (Asumimos escala 30m, puedes cambiarla o pasarla como argumento)
    score, log_score = calcular_score_de_confiabilidad(
        mediana_tsm,
        puntos_calientes_fc,
        puntos_frios_fc,
        escala=30
    )

    # 3. Construir log
    # (El log_score ya dice "Delta: 10.5" o "No hay puntos")
    filename_prefix = nombre_capa.replace(' ', '_').replace('(', '').replace(')', '').replace('-', '_')
    log_msg = f"  -> Capa '{nombre_capa}' procesada. {log_score}"

    # 4. Si la exportación está habilitada, iniciar la tarea
    if EXPORTAR_A_DRIVE:
        export_msg = exportar_imagen_a_drive(
            image=mediana_tsm,       # La imagen ya está clipeada
            bands=['LST_C'],         # Especificamos la banda a exportar
            roi=roi,
            filename=filename_prefix,
            folder=DRIVE_FOLDER
        )
        log_msg += f"\n{export_msg}"

    # 5. Devolver la imagen, nombre, log y EL SCORE
    # El score puede ser un float o None (si no aplica)
    return (mediana_tsm, nombre_capa, log_msg, score)

# GET ROI FROM GDF
def _obtener_roi_desde_gdf(entidad, municipio, gdf): # DONE
    """
    Filtra el GeoDataFrame y convierte la geometría del municipio en un ee.Geometry (ROI).
    Si falla, lanza una Excepción.
    """
    print("--- 2. Creando ROI (ee.Geometry) ---")
    municipio_gdp = gdf[
        (gdf['NOM_ENT'] == entidad) &
        (gdf['NOM_MUN'] == municipio)
    ]

    if municipio_gdp.empty:
        raise Exception(f"No se pudo encontrar la geometría para: {municipio}, {entidad}")

    # Convertimos la geometría de GeoPandas a ee.Geometry
    municipio_gdp_wgs84 = municipio_gdp.to_crs(epsg=4326)
    geom_shapely = municipio_gdp_wgs84.iloc[0].geometry

    try:
        if geom_shapely.geom_type == 'MultiPolygon':
            roi = ee.Geometry.MultiPolygon(geom_shapely.__geo_interface__['coordinates'])
        elif geom_shapely.geom_type == 'Polygon':
            roi = ee.Geometry.Polygon(geom_shapely.__geo_interface__['coordinates'])
        else:
            print("ADVERTENCIA: Geometría no es Polígono ni MultiPolígono. Usando geemap.geopandas_to_ee.")
            roi = geemap.geopandas_to_ee(municipio_gdp_wgs84)

        print("ROI cargada en Earth Engine.")
        return roi

    except Exception as e:
        print(f"ERROR CRÍTICO al convertir Shapely a ee.Geometry: {e}")
        print("Usando el método genérico de geemap como alternativa.")
        try:
            roi = geemap.geopandas_to_ee(municipio_gdp_wgs84)
            print("ROI cargada (método alternativo).")
            return roi
        except Exception as e_alt:
            raise Exception(f"Fallo del método alternativo: {e_alt}")

# GET SATELLITE COLLECTION
def _obtener_coleccion_landsat(roi, fecha_ini, fecha_fin): # DONE
    """
    Filtra la colección de Landsat 8 por fechas, límites (ROI) y nubes.
    Si no encuentra imágenes, lanza una Excepción.
    """
    print("\n--- 4. Filtrando Colección Landsat 8 ---")
    fecha_inicio_str = fecha_ini.strftime('%Y-%m-%d')
    fecha_fin_str = fecha_fin.strftime('%Y-%m-%d')

    coleccion_completa = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterBounds(roi) \
        .filterDate(fecha_inicio_str, fecha_fin_str) \
        .filterMetadata('CLOUD_COVER_LAND', 'less_than', 30)

    count = coleccion_completa.size().getInfo()
    print(f"Imágenes Landsat 8 encontradas en el rango: {count}")

    if count == 0:
        raise Exception("No se encontraron imágenes en el rango y RDI especificados.")

    return coleccion_completa, fecha_inicio_str, fecha_fin_str

# --- 5. FUNCIONES PRINCIPALES (ORQUESTADORES) ---
# Funciones de alto nivel que coordinan a los helpers.

def get_file_path(is_colab_env: bool) -> Path | None:
    """
    Orquesta la obtención de la ruta del archivo, manejando el montaje de Drive.

    Returns:
        Un objeto Path al archivo, o None si la configuración falla.
    """
    if is_colab_env:
        print("Entorno de Google Colab detectado.")

        # 1. Delegar la tarea compleja
        drive_is_ready = _mount_google_drive()

        # 2. Comprobar el resultado
        if not drive_is_ready:
            print("No se puede continuar sin montar Drive.")
            return None

        # 3. Construir la ruta (la tarea principal de esta función)
        return (
            Path(DRIVE_MOUNT_POINT)
            / "Shared drives"
            / SHARED_DRIVE_NAME
            / FOLDER_NAME
            / FILE_NAME
        )

    else:
        # Lógica para entorno local
        print("WARNING: No se detectó Google Colab. Asumiendo archivo local.")
        return Path() / FILE_NAME

def load_and_validate_gdf(file_path: Path) -> gpd.GeoDataFrame | None:
    """
    Orquesta la carga y validación del GeoDataFrame.
    """
    # 1. Delegar la lectura del archivo
    gdf = _read_geodataframe(file_path)

    if gdf is None:
        return None # Falló la lectura, no continuar.

    # 2. Delegar la validación de las columnas
    is_valid = _validate_gdf_columns(gdf)

    if is_valid:
        return gdf
    else:
        return None # Falló la validación

def _procesar_por_periodicidad(coleccion, roi, periodo, start_dt, end_dt, municipio_sel,
                               puntos_calientes_fc, puntos_frios_fc): # <-- Argumentos nuevos
    """
    Toma la colección completa y la procesa según la periodicidad.
    Llama a 'procesar_capa' y al final ordena los resultados por el score.
    """
    print(f"\n--- 5. Procesamiento de TSM por Periodo: {periodo} ---")
    resultados_procesamiento = [] # Lista de tuplas (img, nombre, log, score)

    # --- ¡NUEVO PASO! ---
    # Sanitizamos el nombre del municipio UNA SOLA VEZ
    municipio_export = limpiar_nombre_para_gee(municipio_sel)

    if periodo == 'Mensual':
        # --- PROCESAMIENTO MENSUAL ---
        nombres_meses = ['', 'Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
        current_dt = start_dt.replace(day=1)

        while current_dt <= end_dt:
            current_year = current_dt.year
            current_month = current_dt.month
            days_in_month = calendar.monthrange(current_year, current_month)[1]
            end_filter_dt = current_dt + timedelta(days=days_in_month)

            start_date_str_p = current_dt.strftime('%Y-%m-%d')

            if end_filter_dt > end_dt:
                end_date_str_exclusive = (end_dt + timedelta(days=1)).strftime('%Y-%m-%d')
                nombre_capa = f'TSM_Mediana_{nombres_meses[current_month]}_{current_year}_Parcial_{municipio_export}'
            else:
                end_date_str_exclusive = end_filter_dt.strftime('%Y-%m-%d')
                nombre_capa = f'TSM_Mediana_{nombres_meses[current_month]}_{current_year}_{municipio_export}'

            coleccion_periodo = coleccion.filterDate(start_date_str_p, end_date_str_exclusive)

            if coleccion_periodo.size().getInfo() > 0:
                # --- AQUÍ EL CAMBIO ---
                resultado_tupla = procesar_capa(
                    coleccion_periodo, roi, nombre_capa,
                    puntos_calientes_fc, puntos_frios_fc, # Pasa los puntos
                    False
                )
                resultados_procesamiento.append(resultado_tupla)
            else:
                # (Opcional) Registrar las que se omiten por falta de imágenes
                log_msg = f"  -> Capa '{nombre_capa}' OMITIDA: No hay imágenes en el rango."
                # Añade el resultado con score de "fallido"
                resultados_procesamiento.append((None, nombre_capa, log_msg, -9999.0))

            current_dt = end_filter_dt

    elif periodo == 'Anual':
        # --- PROCESAMIENTO ANUAL ---
        nombre_capa = f'TSM_Mediana_Anual_{start_dt.strftime("%Y-%m-%d")}_a_{end_dt.strftime("%Y-%m-%d")}_{municipio_export}'
        if coleccion.size().getInfo() > 0:
            # --- AQUÍ EL CAMBIO ---
            resultado_tupla = procesar_capa(
                coleccion, roi, nombre_capa,
                puntos_calientes_fc, puntos_frios_fc, # Pasa los puntos
                False
            )
            resultados_procesamiento.append(resultado_tupla)
        else:
            log_msg = f"  -> Capa '{nombre_capa}' OMITIDA: No hay imágenes en el rango."
            resultados_procesamiento.append((None, nombre_capa, log_msg, -9999.0))

    elif periodo == 'Semanal':
        # --- PROCESAMIENTO SEMANAL ---
        current_date = start_dt
        week_index = 1
        while current_date <= end_dt:
            start_week = current_date
            end_week_inclusive = current_date + timedelta(days=6)
            if end_week_inclusive > end_dt:
                end_week_inclusive = end_dt

            start_str_p = start_week.strftime('%Y-%m-%d')
            end_str_exclusive = (end_week_inclusive + timedelta(days=1)).strftime('%Y-%m-%d')
            end_str_inclusive = end_week_inclusive.strftime('%Y-%m-%d')

            coleccion_periodo = coleccion.filterDate(start_str_p, end_str_exclusive)

            if coleccion_periodo.size().getInfo() > 0:
                nombre_capa = f'TSM_Semana_{week_index}_{start_str_p}_a_{end_str_inclusive}_{municipio_export}'
                # --- AQUÍ EL CAMBIO ---
                resultado_tupla = procesar_capa(
                    coleccion_periodo, roi, nombre_capa,
                    puntos_calientes_fc, puntos_frios_fc, # Pasa los puntos
                    False
                )
                resultados_procesamiento.append(resultado_tupla)
            else:
                nombre_capa = f'TSM_Semana_{week_index}_{start_str_p}_a_{end_str_inclusive}_{municipio_export}'
                log_msg = f"  -> Capa '{nombre_capa}' OMITIDA: No hay imágenes en el rango."
                resultados_procesamiento.append((None, nombre_capa, log_msg, -9999.0))

            current_date = current_date + timedelta(days=7)
            week_index += 1
    else:
        raise Exception(f"Periodicidad '{periodo}' no reconocida.")

    # --- 5. RANKING DE RESULTADOS ---
    print("\n--- 6. Ranking de Confiabilidad ---")

    # Separamos las listas por tipo de score
    # x[3] es el score

    # Fallidos (score numérico <= -9000)
    fallidos = [r for r in resultados_procesamiento if isinstance(r[3], (int, float)) and r[3] <= -9000]

    # No Aplicable (score es None, porque no había puntos)
    no_aplicable = [r for r in resultados_procesamiento if r[3] is None]

    # Válidos (score numérico > -9000, o sea, todos los scores calculados)
    validos = [r for r in resultados_procesamiento if isinstance(r[3], (int, float)) and r[3] > -9000]

    # Ordenamos los válidos de MAYOR a MENOR score
    resultados_ordenados = sorted(validos, key=lambda x: x[3], reverse=True)

    print("\n--- Ranking (Mejor a Peor) ---")
    if not resultados_ordenados:
        print("  (No hay resultados con score válido)")
    for (img, nombre, log, score) in resultados_ordenados:
        # El log ya contiene el detalle del score (ej. "Delta: 10.5")
        # Extraemos solo la parte del score del log
        log_score_msg = log.split(' procesada. ')[-1]
        print(f"  Score: {score:7.2f} | {log_score_msg} | Capa: {nombre}")

    print("\n--- Sin Calificación (No Aplicable) ---")
    if not no_aplicable:
        print("  (No hay resultados sin calificación)")
    for (img, nombre, log, score) in no_aplicable:
        log_score_msg = log.split(' procesada. ')[-1]
        print(f"  Score: N/A  | {log_score_msg} | Capa: {nombre}")

    print("\n--- Fallidos u Omitidos ---")
    if not fallidos:
        print("  (No hay resultados fallidos)")
    for (img, nombre, log, score) in fallidos:
        # El log original ya dice "OMITIDA" o "Error"
        print(f"  {log}")

    return resultados_procesamiento

# DATA PROCESSING
def _agregar_capa_color_verdadero(coleccion, roi, fecha_ini_str, fecha_fin_str, municipio_sel, exportar, carpeta_drive):
    """
    Genera la capa de Color Verdadero y la devuelve.
    NO la añade al mapa.
    """
    print("\n--- 8. Generando Capa de Color Verdadero ---")

    # 1. Clipar la imagen una sola vez
    composicion_anual_sr = coleccion.map(enmascarar_nubes_scala).median().clip(roi)

    vis_params_color_verdadero = {
        'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0.0, 'max': 0.5, 'gamma': 1.4,
    }

    # --- ¡NUEVO PASO! ---
    # Sanitizamos el nombre del municipio UNA SOLA VEZ
    municipio_export = limpiar_nombre_para_gee(municipio_sel)

    nombre_ref = f'Color_Verdadero_Mediana_{fecha_ini_str}_a_{fecha_fin_str}_{municipio_export}'

    # 2. (¡SE ELIMINA LA LÍNEA 'controles["Map"].addLayer'!)

    log_msg = f"Capa de Color Verdadero procesada."
    print(log_msg)

    # 3. Lógica de exportación
    if exportar:
        filename_prefix = nombre_ref.replace(' ', '_').replace('(', '').replace(')', '').replace('-', '_')

        export_msg = exportar_imagen_a_drive(
            image=composicion_anual_sr,
            bands=['SR_B4', 'SR_B3', 'SR_B2'],
            roi=roi,
            filename=filename_prefix,
            folder=carpeta_drive
        )
        log_msg += f"\n{export_msg}"
        print(export_msg)

    # 4. Devolver la imagen, su nombre, sus params de vis y el log
    return (composicion_anual_sr, nombre_ref, vis_params_color_verdadero, log_msg)

# --- 6. FUNCIONES PRINCIPALES (MAIN) ---
# MAIN CALL FOR SET UP GEODATA
def setup_geodata() -> gpd.GeoDataFrame | None:
    """
    Función principal para orquestar todo el proceso de carga.
    1. Revisa el entorno.
    2. Obtiene la ruta del archivo.
    3. Carga y valida el archivo.
    """
    print("--- Iniciando configuración de datos ---")
    is_colab = check_is_colab()
    file_path = get_file_path(is_colab)

    if file_path:
        gdf = load_and_validate_gdf(file_path)
        return gdf
    else:
        print("\nNo se pudo determinar la ruta del archivo. No se puede cargar el GeoDataFrame.")
        return None

# MAIN CALL FOR AUTHENTICATE GEE
def initialize_gee(project_id: str) -> bool:
    """
    Autentica (si es necesario) e inicializa Google Earth Engine.

    Returns:
        bool: True si GEE se inicializó con éxito, False si falló.
    """
    try:
        # 1. Intento optimista:
        # Intenta inicializar usando credenciales existentes (cacheadas).
        print("Intentando inicializar Earth Engine...")
        ee.Initialize(project=project_id) # 1. Fails
        print("¡Earth Engine inicializado exitosamente (usando credenciales existentes)!")
        return True

    except Exception as e:
        # 2. Si falla, es probable que no haya credenciales.
        print(f"Inicialización fallida ({e}). Se requiere autenticación.")

        try:
            # 3. Forzar la autenticación (esto abrirá una ventana/prompt)
            print("Por favor, sigue las instrucciones de autenticación...")
            ee.Authenticate() # 3. User clicks "Cancel"

            # 4. Reintentar la inicialización después de autenticar
            ee.Initialize(project=project_id) # 4. Fails *again*
            print("¡Earth Engine autenticado e inicializado exitosamente!")
            return True

        except Exception as auth_e: # We catch the 2nd failure!
            # 5. Si la autenticación o el segundo intento fallan
            print(f"ERROR: No se pudo autenticar o inicializar GEE: {auth_e}")
            return False

# MAIN CALL TO CREATE CONTROLS
def crear_controles(gdf, default_drive_folder):
    """
    Crea y agrupa todos los widgets de la interfaz de usuario.

    Args:
        gdf (gpd.GeoDataFrame): GeoDataFrame con la información de entidades y municipios.
        default_drive_folder (str): Ruta por defecto para la carpeta de Drive.

    Returns:
        dict: Un diccionario que contiene todos los widgets interactivos.
    """

    # --- 1. Crear el Mapa ---
    Map = geemap.Map(center=[20.5888, -100.3899], zoom=7)

    # --- 2. Controles de Ubicación ---
    if gdf is not None:
        entidades_unicas = sorted(gdf['NOM_ENT'].unique())
    else:
        entidades_unicas = ['ERROR: GeoJSON no cargado']

    entidad_dropdown = widgets.Dropdown(
        options=entidades_unicas,
        description='1. Entidad:',
        disabled=(gdf is None),
        style={'description_width': 'initial'}
    )

    municipio_dropdown = widgets.Dropdown(
        options=[],
        description='2. Municipio:',
        disabled=(gdf is None),
        style={'description_width': 'initial'}
    )

    # --- 3. Contenedor de Salida (Necesario para el observador) ---
    output = widgets.Output()

    # --- 4. Observador (Definido DENTRO) ---
    def on_entidad_change(change):
        if change['type'] == 'change' and change['name'] == 'value' and gdf is not None:
            entidad_seleccionada = change['new']
            municipios_df = gdf[gdf['NOM_ENT'] == entidad_seleccionada]
            municipios_unicos = sorted(municipios_df['NOM_MUN'].unique())
            municipio_dropdown.options = municipios_unicos
            municipio_dropdown.disabled = False
            with output:
                clear_output()
                #print(f"Municipios disponibles para {entidad_seleccionada} cargados.")

    # Conectar el observador
    entidad_dropdown.observe(on_entidad_change)

    # --- 5. Controles de Temporalidad y Exportación ---
    # (El código de fecha_inicial_picker, fecha_final_picker,
    # periodicidad_radio, y copy_path_textbox va aquí...
    # Lo omito para que sea breve, pero NO lo borres de tu código)

    fecha_inicial_picker = widgets.DatePicker(
        value=date(2023, 1, 1),
        description='3. Fecha Inicial:',
        disabled=False,
        style={'description_width': 'initial'}
    )

    fecha_final_picker = widgets.DatePicker(
        value=date(2023, 12, 31),
        description='4. Fecha Final:',
        disabled=False,
        style={'description_width': 'initial'}
    )

    periodicidad_radio = widgets.RadioButtons(
        options=['Semanal', 'Mensual', 'Anual'],
        description='5. Periodicidad:',
        value='Mensual',
        disabled=False,
        style={'description_width': 'initial'}
    )

    copy_path_textbox = widgets.Text(
        value=default_drive_folder,
        description='6. Carpeta en Drive:',
        placeholder='Ej: GEE_Resultados/Mi_Proyecto',
        disabled=False,
        layout=widgets.Layout(width='auto'),
        style={'description_width': 'initial'}
    )

    # --- 5. Control de Puntos (NUEVO) ---
    upload_points_button = widgets.Button(
        description="Subir CSV de Puntos (caliente/frío)",
        button_style='primary', # Un color azul
        icon='upload'
    )

    # --- 6. Botón de Ejecución ---
    ejecutar_button = widgets.Button(
        description="Ejecutar Visualización GEE",
        button_style='success',
        icon='earth'
    )

    # --- 7. INICIALIZACIÓN (La mejor solución) ### NUEVO ### ---
    # Inicializar el dropdown de municipio con la primera entidad
    # para que no esté vacío al cargar.
    # Todas las variables (entidades_unicas, on_entidad_change)
    # ya existen en este contexto.
    if entidades_unicas and entidades_unicas[0] != 'ERROR: GeoJSON no cargado':
        on_entidad_change({'type': 'change', 'name': 'value', 'new': entidades_unicas[0]})
    # --- Fin de la sección nueva ---


    # --- 8. Devolver todo en un diccionario ---
    widgets_dict = {
        "Map": Map,
        "entidad": entidad_dropdown,
        "municipio": municipio_dropdown,
        "fecha_ini": fecha_inicial_picker,
        "fecha_fin": fecha_final_picker,
        "periodicidad": periodicidad_radio,
        "ruta_copia": copy_path_textbox,
        "upload_points_button": upload_points_button,
        "boton": ejecutar_button,
        "output": output
    }

    return widgets_dict


# MAIN CALL FOR CONTROL POINTS
# --- 7. HANDLERS PARA WIDGETS ---
def on_upload_points_click(b):
    """
    Manejador para el botón 'Subir CSV de Puntos'.
    Lee un CSV, crea ee.Features y los almacena en el dict 'controles'
    como dos FeatureCollections: 'hot_points_fc' y 'cold_points_fc'.
    """
    with controles["output"]:
        clear_output()
        print("--- Subiendo Archivo de Puntos de Control ---")
        print("Por favor, selecciona tu archivo .csv (columnas: type, lon, lat)")

        try:
            # 1. Usar el cargador de archivos de Colab
            uploaded = files.upload()

            if not uploaded:
                print("Carga cancelada.")
                return

            # 2. Obtener el nombre y contenido del primer archivo subido
            filename = next(iter(uploaded))
            content = uploaded[filename].decode('utf-8')
            print(f"Archivo '{filename}' cargado. Procesando...")

            # 3. Usar 'io.StringIO' para tratar el string como un archivo
            f = io.StringIO(content)
            reader = csv.DictReader(f)

            # --- MANEJO DE MÚLTIPLES PUNTOS ---
            # Creamos listas para agrupar los puntos por tipo
            hot_features_list = []
            cold_features_list = []

            # 4. Iterar sobre las filas y crear las 'Features' de GEE
            for row in reader:
                try:
                    # Asumimos columnas: 'type', 'lon', 'lat'
                    tipo = row['type'].strip().lower()
                    lon = float(row['lon'])
                    lat = float(row['lat'])

                    point_geom = ee.Geometry.Point([lon, lat])

                    # Creamos una 'Feature' (Geometría + Propiedades)
                    feature = ee.Feature(point_geom, {'type': tipo})

                    if tipo == 'hot':
                        hot_features_list.append(feature)
                    elif tipo == 'cold':
                        cold_features_list.append(feature)

                except Exception as e:
                    print(f"  -> ADVERTENCIA: No se pudo leer la fila {row}. Error: {e}")

            # 5. Guardar las listas como FeatureCollections en 'controles'
            controles['hot_points_fc'] = None
            controles['cold_points_fc'] = None

            if hot_features_list:
                controles['hot_points_fc'] = ee.FeatureCollection(hot_features_list)
                print(f"  -> {len(hot_features_list)} Puntos Calientes cargados.")
                controles["Map"].addLayer(controles['hot_points_fc'], {'color': 'FF0000'}, 'Puntos Calientes')

            if cold_features_list:
                controles['cold_points_fc'] = ee.FeatureCollection(cold_features_list)
                print(f"  -> {len(cold_features_list)} Puntos Fríos cargados.")
                controles["Map"].addLayer(controles['cold_points_fc'], {'color': '0000FF'}, 'Puntos Fríos')

            if not hot_features_list and not cold_features_list:
                 print("ERROR: No se cargaron puntos válidos. Asegúrate que el CSV tenga columnas 'type', 'lon', 'lat' y que 'type' sea 'hot' o 'cold'.")
            else:
                print("¡Puntos de control listos!")

        except Exception as e:
            print(f"\nERROR CRÍTICO durante la carga del archivo: {e}")

def _validar_puntos_con_roi(roi, controles):
    """
    Toma los puntos '_TODOS' del dict 'controles', los filtra con el 'roi'
    y guarda el resultado en las claves de producción
    ('puntos_calientes_fc' y 'puntos_frios_fc').
    """
    print("\n--- 3. Validando Puntos de Control con ROI ---")

    # Obtener los puntos cargados (TODOS)
    hot_fc_todos = controles.get('hot_points_fc')
    cold_fc_todos = controles.get('cold_points_fc')

    if not hot_fc_todos and not cold_fc_todos:
        print("  -> No se subieron puntos de control. El score no se calculará.")
        controles['valid_hot_points'] = None
        controles['valid_cold_points'] = None
        return True # No es un error, es un caso de uso

    hot_fc_validos = None
    cold_fc_validos = None

    # --- Validar Calientes ---
    if hot_fc_todos:
        hot_fc_validos = hot_fc_todos.filterBounds(roi)
        # Guardar la colección validada para el procesamiento
        controles['valid_hot_points'] = hot_fc_validos

        # Reportar y añadir al mapa
        hot_valid_count = hot_fc_validos.size().getInfo()
        print(f"  -> Puntos Calientes VÁLIDOS (dentro del ROI): {hot_valid_count}")
        if hot_valid_count > 0:
            controles["Map"].addLayer(hot_fc_validos, {'color': 'FFFFFF', 'pointShape': 'X'}, 'Puntos Calientes (Validados)')

    # --- Validar Fríos ---
    if cold_fc_todos:
        cold_fc_validos = cold_fc_todos.filterBounds(roi)
        # Guardar la colección validada para el procesamiento
        controles['valid_cold_points'] = cold_fc_validos

        # Reportar y añadir al mapa
        cold_valid_count = cold_fc_validos.size().getInfo()
        print(f"  -> Puntos Fríos VÁLIDOS (dentro del ROI): {cold_valid_count}")
        if cold_valid_count > 0:
            controles["Map"].addLayer(cold_fc_validos, {'color': 'FFFFFF', 'pointShape': 'X'}, 'Puntos Fríos (Validados)')

    # --- Chequeo final ---
    total_validos = (hot_fc_validos.size().getInfo() if hot_fc_validos else 0) + \
                    (cold_fc_validos.size().getInfo() if cold_fc_validos else 0)

    if total_validos == 0 and (hot_fc_todos or cold_fc_todos):
        print("\nADVERTENCIA: Se cargaron puntos, pero NINGUNO está dentro del ROI seleccionado.")

    return True

# MAIN CALL FOR RUNNING ANALYSIS
def ejecutar_analisis_gee(b):
    """
    Función que se ejecuta al presionar el botón:
    Accede a los widgets a través del diccionario 'controles'.
    """

    # 1. Limpiar salida y Recolectar valores
    # --- ACCEDER A 'output' DESDE EL DICCIONARIO ---
    with controles["output"]:
        clear_output()
        print("--- 1. Iniciando Proceso ---")

        # --- ACCEDER A LOS WIDGETS DESDE EL DICCIONARIO ---
        entidad_sel = controles["entidad"].value
        municipio_sel = controles["municipio"].value
        fecha_ini_sel = controles["fecha_ini"].value
        fecha_fin_sel = controles["fecha_fin"].value
        periodo_sel = controles["periodicidad"].value
        ruta_copia_secundaria = controles["ruta_copia"].value.strip() # <-- ¡AQUÍ ESTÁ TU CAMBIO!

        # Validar entradas
        if not all([entidad_sel, municipio_sel, fecha_ini_sel, fecha_fin_sel]):
            print("ERROR: Por favor, asegúrate de seleccionar Entidad, Municipio, Fecha Inicial y Fecha Final.")
            return

        if fecha_fin_sel < fecha_ini_sel:
              print("ERROR: La Fecha Final no puede ser anterior a la Fecha Inicial.")
              return

        print(f"Ubicación: {municipio_sel}, {entidad_sel}")
        print(f"Rango: {fecha_ini_sel} a {fecha_fin_sel}")
        print(f"Periodicidad: {periodo_sel}")
        print(f"Exportación GEE (Carpeta principal): {'Sí' if EXPORTAR_A_DRIVE else 'No'} -> '{DRIVE_FOLDER}'")

        # Advertencias de copia (lógica de UI)
        if EXPORTAR_A_DRIVE and not ruta_copia_secundaria:
            print("NOTA: La exportación principal está activa, pero la ruta de copia secundaria está vacía.")
        elif ruta_copia_secundaria and not EXPORTAR_A_DRIVE:
            print("ADVERTENCIA: Se especificó una ruta de copia, pero 'EXPORTAR_A_DRIVE' es False. Se omitirá.")


        # --- INICIO DE LÓGICA DE PROCESAMIENTO ---
        try:
            # 2. Obtener ROI (Paso de Geometría)
            # 'gdf' es global, así que está bien
            roi = _obtener_roi_desde_gdf(entidad_sel, municipio_sel, gdf)

            # --- ACCEDER AL 'Map' DESDE EL DICCIONARIO ---
            controles["Map"].centerObject(roi, 9)
            controles["Map"].addLayer(roi, {'color': 'red'}, f'RDI: {municipio_sel}', True)

            # --- 3. ¡VALIDACIÓN DE PUNTOS! (EL NUEVO PASO) ---
            # Esta función usa el 'roi' que acabamos de crear para filtrar
            # los puntos que se cargaron previamente.
            _validar_puntos_con_roi(roi, controles)


            # 3. Obtener Colección (Paso de Datos GEE)
            coleccion, fecha_ini_str, fecha_fin_str = _obtener_coleccion_landsat(roi, fecha_ini_sel, fecha_fin_sel)

            # 4. Procesar TSM (Llama a la función desacoplada)
            # 'resultados_tsm' es una lista de tuplas: [(img, nombre, log), ...]

            puntos_calientes_validados = controles.get('valid_hot_points')
            puntos_frios_validados = controles.get('valid_cold_points')

            resultados_tsm = _procesar_por_periodicidad(
                coleccion,
                roi,
                periodo_sel,
                fecha_ini_sel, fecha_fin_sel,
                municipio_sel,
                puntos_calientes_validados,  # Pasa la colección validada
                puntos_frios_validados   # Pasa la colección validada
            )

            # --- 5. AÑADIR CAPAS TSM AL MAPA ---
            print("\n--- 7. Añadiendo capas TSM al mapa ---")
            exportacion_iniciada = False
            for (imagen, nombre, log, score) in resultados_tsm:
                print(log) # Imprimir el log (que ya tiene el score)
                if imagen: # Solo añadir si la imagen no es None (es decir, si no se omitió)
                    controles["Map"].addLayer(imagen, VIS_PARAMS_TSM, nombre, False) # Añadir capa (Visibilidad False por defecto)
                if "EXPORTACIÓN INICIADA" in log:
                    exportacion_iniciada = True

            # (Opcional) Mensaje de exportación
            if EXPORTAR_A_DRIVE and exportacion_iniciada:
                 print("\n" + "="*50)
                 print("  ¡ATENCIÓN! EXPORTACIONES AUTOMÁTICAS INICIADAS")
                 print(f"  Los archivos aparecerán en 'My Drive' / '{DRIVE_FOLDER}'")
                 print("  Puede monitorear el progreso en la pestaña 'Tasks' de GEE.")
                 print("="*50)

            # --- 6. PROCESAR Y AÑADIR CAPA DE REFERENCIA ---
            (img_cv, nombre_cv, vis_cv, log_cv) = _agregar_capa_color_verdadero(
                coleccion, roi,
                fecha_ini_str, fecha_fin_str,
                municipio_sel,
                EXPORTAR_A_DRIVE, DRIVE_FOLDER
            )

            # Añadir la capa de color verdadero al mapa
            controles["Map"].addLayer(img_cv, vis_cv, nombre_cv, True) # Visibilidad True

            #display(controles["Map"]) # Muestra el mapa

        except Exception as e:
            # Capturamos cualquier error de las funciones auxiliares
            print(f"\n--- ERROR DURANTE LA EJECUCIÓN ---")
            print(f"Error: {e}")
            print("El proceso se ha detenido.")
            return



### 2. Autenticación (Drive y GEE)

In [ ]:
# Once
gdf = setup_geodata()
gee_ready = initialize_gee(GEE_PROJECT_ID)

## Ejecución herramienta


### 1. Controles y visualización

In [ ]:
# @title
# EXECUTION
if (gdf is not None) and gee_ready:
    # --- 8. CREACIÓN Y VISUALIZACIÓN DE LA UI ---

    controles = crear_controles(gdf, DRIVE_FOLDER)

    # --- INICIALIZACIÓN DE PUNTOS DE CONTROL ---
    # Los inicializamos como 'None'. La función de carga los llenará.
    controles['hot_points_fc'] = None
    controles['cold_points_fc'] = None

    # --- CONEXIÓN DE HANDLERS ---
    controles['upload_points_button'].on_click(on_upload_points_click) # <-- AQUÍ
    controles['boton'].on_click(ejecutar_analisis_gee)

    # --- Organización de la UI (Actualizada) ---
    controles_ui = widgets.VBox([
        widgets.HBox([controles['entidad'], controles['municipio']]),
        widgets.HBox([controles['fecha_ini'], controles['fecha_fin']]),
        controles['periodicidad'],
        controles['ruta_copia'],
        controles['upload_points_button'],
        controles['boton'],
        controles['output']
    ])

    # Mostrar el mapa y los controles
    display(controles['Map'])
    display(controles_ui)

else:
    print("\n--- La carga de datos falló. Revisa los mensajes de error. ---")
